In [1]:
import tensorflow.compat.v1 as tf
import librosa
import pyaudio
import wave
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
tf.disable_v2_behavior()

import os
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = int(RATE / 10)
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"
DATA_PATH = "./data/"
X_train = []
X_test = []
Y_train = []
Y_test = []
tf_classes = 0
def load_wave_generator(path):    #음성 데이터 npy로 변경

    batch_waves = []
    labels = []
    X_data = []
    Y_label = []
    global X_train, X_test, Y_train, Y_test, tf_classes

    folders = os.listdir(path)

    for folder in folders:
        if not os.path.isdir(path):continue 
        files = os.listdir(path+"/"+folder)
        print("Foldername :",folder,"-",len(files),"files")
        for wav in files:
            if not wav.endswith(".wav"):continue
            else:
                y, sr = librosa.load(path+"/"+folder+"/"+wav)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

                X_data.extend(mfcc)
               # print(len(mfcc))

                label = [0 for i in range(len(folders))]
                label[tf_classes] = 1

                for i in range(len(mfcc)):
                    Y_label.append(label)
                #print(Y_label)
        tf_classes = tf_classes+1
    #end loop
    print("X_data :",np.shape(X_data))
    print("Y_label :",np.shape(Y_label))
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_data), np.array(Y_label))

    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./data.npy",xy)

load_wave_generator(DATA_PATH)

#t = np.array(X_train);
#print("!!!!!!!!",t,t.shape,X_train)
print(tf_classes,"classes!!")
print("X_train :",np.shape(X_train))
print("Y_train :",np.shape(Y_train))
print("X_test :",np.shape(X_test))
print("Y_test :",np.shape(Y_test))
####################
#clf = LogisticRegression()
#clf.fit(X_train, Y_train)
####################



Instructions for updating:
non-resource variables are not supported in the long term
Foldername : 0 - 20 files
Foldername : 1 - 20 files
Foldername : 2 - 20 files
Foldername : 3 - 20 files
Foldername : 4 - 20 files
Foldername : 5 - 20 files
X_data : (60240, 13)
Y_label : (60240, 6)
6 classes!!
X_train : (45180, 13)
Y_train : (45180, 6)
X_test : (15060, 13)
Y_test : (15060, 6)


C:\Users\alexr\anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [3]:
X_train, X_test, Y_train, Y_test = np.load("./data.npy", allow_pickle=True)#allow_pickle add
X_train = X_train.astype("float")
X_test = X_test.astype("float")

tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(777)
learning_rate = 0.001
training_epochs = 100
keep_prob = tf.compat.v1.placeholder(tf.float32)
sd = 1 / np.sqrt(13) 

X = tf.compat.v1.placeholder(tf.float32, [None, 13])
#
Y = tf.compat.v1.placeholder(tf.float32, [None, tf_classes])

# W = tf.Variable(tf.random_normal([216, 200]))
# b = tf.Variable(tf.random_normal([200]))

#1st hidden layer
W1 = tf.compat.v1.get_variable("w1",
    #tf.random_normal([216, 180], mean=0, stddev=sd),
        shape=[13, 256],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b1 = tf.Variable(tf.random.normal([256], mean=0, stddev=sd), name="b1")
L1 = tf.nn.relu(tf.matmul(X, W1) + b1) 
L1 = tf.nn.dropout(L1, rate = 1 - (keep_prob))

# 2nd hidden layer
W2 = tf.compat.v1.get_variable("w2",
    #tf.random_normal([180, 150], mean=0, stddev=sd),
         shape=[256, 256],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b2 = tf.Variable(tf.random.normal([256], mean=0, stddev=sd), name="b2")
L2 = tf.nn.tanh(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, rate = 1 - (keep_prob))

# 3th hidden layer
W3 = tf.compat.v1.get_variable("w3",
    #tf.random_normal([150, 100], mean=0, stddev=sd),
            shape=[256, 256],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b3 = tf.Variable(tf.random.normal([256], mean=0, stddev=sd), name="b3")
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, rate = 1 - (keep_prob))

# 4th hidden layer
W4 = tf.compat.v1.get_variable("w4",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[256, 128],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b4 = tf.Variable(tf.random.normal([128], mean=0, stddev=sd), name="b4")
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, rate = 1 - (keep_prob))

# 5th hidden layer
W5 = tf.compat.v1.get_variable("w5",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[128, 128],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b5 = tf.Variable(tf.random.normal([128], mean=0, stddev=sd), name="b5")
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)
L5 = tf.nn.dropout(L5, rate = 1 - (keep_prob))

# 6 hidden layer
W6 = tf.compat.v1.get_variable("w6",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[128, 128],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b6 = tf.Variable(tf.random.normal([128], mean=0, stddev=sd), name="b6")
L6 = tf.nn.relu(tf.matmul(L5, W6) + b6) 
L6 = tf.nn.dropout(L6, rate = 1 - (keep_prob))

# 7th hidden layer
W7 = tf.compat.v1.get_variable("w7",
    #tf.random_normal([100, 50], mean=0, stddev=sd),
             shape=[128, 128],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b7 = tf.Variable(tf.random.normal([128], mean=0, stddev=sd), name="b7")
L7 = tf.nn.relu(tf.matmul(L6, W7) + b7)
L7 = tf.nn.dropout(L7, rate = 1 - (keep_prob))

# final layer
W8 = tf.compat.v1.get_variable("w8",
    #tf.random_normal([50, tf_classes], mean=0, stddev=sd),
            shape=[128, tf_classes],
                     initializer=tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"))
b8 = tf.Variable(tf.random.normal([tf_classes], mean=0, stddev=sd), name="b8")
hypothesis = tf.matmul(L7, W8) + b8



#cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
cost = tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=tf.stop_gradient(Y)))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))


batch_size=1
x_len = len(X_train)
#*2
if(x_len%2==0):
    batch_size = 2
elif(x_len%3==0):
    batch_size = 3
elif(x_len%4==0):
    batch_size = 4
else:
    batch_size = 1

split_X = np.split(X_train,batch_size)
split_Y = np.split(Y_train,batch_size)

sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    for i in range(batch_size):
        batch_xs = split_X[i]
        batch_ys = split_Y[i]
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / batch_size
        #if(epoch%10==0):
    print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))

print('Learning Finished!')

Epoch: 0000 cost = 2.064874768
Epoch: 0001 cost = 1.868339419
Epoch: 0002 cost = 1.838817835
Epoch: 0003 cost = 1.824440539
Epoch: 0004 cost = 1.817013144
Epoch: 0005 cost = 1.811315775
Epoch: 0006 cost = 1.807290137
Epoch: 0007 cost = 1.802938223
Epoch: 0008 cost = 1.800034583
Epoch: 0009 cost = 1.797730863
Epoch: 0010 cost = 1.796716869
Epoch: 0011 cost = 1.791337371
Epoch: 0012 cost = 1.785406113
Epoch: 0013 cost = 1.779751837
Epoch: 0014 cost = 1.768437564
Epoch: 0015 cost = 1.756364286
Epoch: 0016 cost = 1.741080761
Epoch: 0017 cost = 1.723903716
Epoch: 0018 cost = 1.703667283
Epoch: 0019 cost = 1.683926702
Epoch: 0020 cost = 1.665920258
Epoch: 0021 cost = 1.648683906
Epoch: 0022 cost = 1.629659593
Epoch: 0023 cost = 1.611580372
Epoch: 0024 cost = 1.599230587
Epoch: 0025 cost = 1.583233297
Epoch: 0026 cost = 1.561247647
Epoch: 0027 cost = 1.547543406
Epoch: 0028 cost = 1.529516578
Epoch: 0029 cost = 1.515975118
Epoch: 0030 cost = 1.496094346
Epoch: 0031 cost = 1.479024708
Epoch: 0

In [4]:
for epoch in range(training_epochs):
    avg_cost = 0
    for i in range(batch_size):
        batch_xs = split_X[i]
        batch_ys = split_Y[i]
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / batch_size
        #if(epoch%10==0):
    print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))

print('Learning Finished!')


Epoch: 0000 cost = 0.934049457
Epoch: 0001 cost = 0.924213469
Epoch: 0002 cost = 0.920375049
Epoch: 0003 cost = 0.916062355
Epoch: 0004 cost = 0.907304972
Epoch: 0005 cost = 0.908597350
Epoch: 0006 cost = 0.906613648
Epoch: 0007 cost = 0.901054263
Epoch: 0008 cost = 0.892286867
Epoch: 0009 cost = 0.890569240
Epoch: 0010 cost = 0.887806118
Epoch: 0011 cost = 0.882237166
Epoch: 0012 cost = 0.877040356
Epoch: 0013 cost = 0.871224612
Epoch: 0014 cost = 0.869587451
Epoch: 0015 cost = 0.869379252
Epoch: 0016 cost = 0.863109887
Epoch: 0017 cost = 0.855811030
Epoch: 0018 cost = 0.857121617
Epoch: 0019 cost = 0.856333256
Epoch: 0020 cost = 0.851510882
Epoch: 0021 cost = 0.850098401
Epoch: 0022 cost = 0.847107917
Epoch: 0023 cost = 0.840172261
Epoch: 0024 cost = 0.839311540
Epoch: 0025 cost = 0.837675035
Epoch: 0026 cost = 0.834271103
Epoch: 0027 cost = 0.831058562
Epoch: 0028 cost = 0.824958831
Epoch: 0029 cost = 0.823845744
Epoch: 0030 cost = 0.820797205
Epoch: 0031 cost = 0.814559281
Epoch: 0

In [5]:
saver = tf.compat.v1.train.Saver()
saver.save(sess, './my_voice_model2')

'./my_voice_model2'

In [7]:
y, sr = librosa.load("./test_이재은.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(6)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))


#correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))
#print("Label :",sess.run(tf.argmax(Y_test,1)))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("predict")
print(pd.value_counts(pd.Series(sess.run(tf.argmax(input=hypothesis, axis=1),
                                    feed_dict={X: X_test, keep_prob:1}))))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))


(502, 13)
(502, 6)
predict
2    488
1      4
3      4
5      4
4      2
dtype: int64
Accuracy:  0.9721116


In [8]:
y, sr = librosa.load("./test_문재인대통령.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(6)]#class가 3개이니까 y_test만드는 과정
label[4] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))


#correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))
#print("Label :",sess.run(tf.argmax(Y_test,1)))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("predict")
print(pd.value_counts(pd.Series(sess.run(tf.argmax(input=hypothesis, axis=1),
                                    feed_dict={X: X_test, keep_prob:1}))))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))


(502, 13)
(502, 6)
predict
4    354
3     42
0     37
2     31
1     29
5      9
dtype: int64
Accuracy:  0.7051793


In [9]:
y, sr = librosa.load("./test_유인나1.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(6)]#class가 3개이니까 y_test만드는 과정
label[0] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))


#correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))
#print("Label :",sess.run(tf.argmax(Y_test,1)))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("predict")
print(pd.value_counts(pd.Series(sess.run(tf.argmax(input=hypothesis, axis=1),
                                    feed_dict={X: X_test, keep_prob:1}))))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))


(502, 13)
(502, 6)
predict
0    468
2     15
4     13
1      5
3      1
dtype: int64
Accuracy:  0.93227094


In [10]:
y, sr = librosa.load("./test_유인나2.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(6)]#class가 3개이니까 y_test만드는 과정
label[0] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))


#correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))
#print("Label :",sess.run(tf.argmax(Y_test,1)))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("predict")
print(pd.value_counts(pd.Series(sess.run(tf.argmax(input=hypothesis, axis=1),
                                    feed_dict={X: X_test, keep_prob:1}))))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))


(502, 13)
(502, 6)
predict
0    441
5     37
1     12
3      9
2      2
4      1
dtype: int64
Accuracy:  0.87848604


In [11]:
y, sr = librosa.load("./test_배철수.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(6)]#class가 3개이니까 y_test만드는 과정
label[1] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))


#correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))
#print("Label :",sess.run(tf.argmax(Y_test,1)))

correct_prediction = tf.equal(tf.argmax(input=hypothesis, axis=1), tf.argmax(input=Y, axis=1))
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
print("predict")
print(pd.value_counts(pd.Series(sess.run(tf.argmax(input=hypothesis, axis=1),
                                    feed_dict={X: X_test, keep_prob:1}))))
print("Accuracy: ", sess.run(accuracy, feed_dict={X: X_test, Y:Y_test, keep_prob:1}))


(502, 13)
(502, 6)
predict
1    416
3     26
2     20
4     20
0     14
5      6
dtype: int64
Accuracy:  0.8286853
